In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
import hyperspy.api as hs

In [2]:
dpcu0 = hs.load("Data/CuTwinTilt/20220305_154456/SPED_256x64x12_25x25_scanRot340_NBD_alpha5_spot0p5nm_CL12cm_1deg_100Hz_20ms.hspy")

ERROR:numba.cuda.cudadrv.driver:Call to cuInit results in UNKNOWN_CUDA_ERROR
/home/joseph/miniconda3/envs/pyxemmaster/lib/python3.9/site-packages/hyperspy/misc/utils.py:470: VisibleDeprecationWarning: Use of the `binned` attribute in metadata is going to be deprecated in v2.0. Set the `axis.is_binned` attribute instead. 
  warnings.warn('Use of the `binned` attribute in metadata is '
/home/joseph/miniconda3/envs/pyxemmaster/lib/python3.9/site-packages/hyperspy/io.py:524: VisibleDeprecationWarning: Loading old file version. The binned attribute has been moved from metadata.Signal to axis.is_binned. Setting this attribute for all signal axes instead.
  warnings.warn('Loading old file version. The binned attribute '


In [ ]:
dpcu0.center_direct_beam(method = 'blur',half_square_width=15,return_shifts=True, sigma = 3)

In [8]:
dpcu0.plot(vmax = 200)

In [ ]:
dpcu0.subtract_diffraction_background()

In [ ]:
# dpcu0.isig[108:148,108:148] = 0

In [3]:
from skimage import filters

def crop_minimum(image, minimum=3):
    copied = image.copy()
    copied[copied <= minimum] = 0.
    return copied

In [4]:
procdat = dpcu0.inav[196,22].subtract_diffraction_background(method="difference of gaussians",
                                                            min_sigma=7,
                                                            max_sigma=10, 
                                                            lazy_result=False)
procdat = procdat.map(filters.gaussian, sigma=0.5, inplace=False)
procdat = procdat.map(crop_minimum, minimum = 1, inplace=False)
procdat = procdat.map(filters.gaussian, sigma=0.5, inplace=False)

[########################################] | 100% Completed |  0.1s


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
procdat.plot(vmax = 6, scalebar = False, cmap = 'inferno_r')

In [ ]:
procdat = dpcu0.subtract_diffraction_background(method="difference of gaussians",
                                                            min_sigma=8,
                                                            max_sigma=9, 
                                                            lazy_result=False)
procdat = procdat.map(filters.gaussian, sigma=1, inplace=False)
procdat = procdat.map(crop_minimum, minimum = 0.4, inplace=False)

In [ ]:
procdat.plot(vmax = 10)

In [ ]:
dpcu0 = procdat

In [ ]:
procdat.save('Data/CuTwinTilt/SPEDCutilt2_bgred.hspy')

In [ ]:
dpcu0 = hs.load("Data/CuTwinTilt/SPEDCutilt2_bgred.hspy")

In [ ]:
resolution = 0.5
grid_cub = get_beam_directions_grid("cubic", resolution, mesh="spherified_cube_edge")

structure_matrix = diffpy.structure.loadStructure('Data/Cu_mp-30_conventional_standard.cif')
half_shape = (256//2, 256//2)

reciprocal_radius = np.sqrt(half_shape[0]**2 + half_shape[1]**2)*diffraction_calibration

diff_gen = DiffractionGenerator(accelerating_voltage=200,
                                precession_angle=1,
                                scattering_params=None,
                                shape_factor_model="linear",
                                minimum_intensity=0.1,
                                )

lib_gen = DiffractionLibraryGenerator(diff_gen)

library_phases = StructureLibrary(["Cu"], [structure_matrix], [grid_cub])

diff_lib = lib_gen.get_diffraction_library(library_phases,
                                           calibration=diffraction_calibration,
                                           reciprocal_radius=reciprocal_radius,
                                           half_shape=half_shape,
                                           with_direct_beam=False,
                                           max_excitation_error=0.06)

In [ ]:
def template_log_func(x):
    return np.log10(x + 0.01)
import diffpy
from diffsims.libraries.structure_library import StructureLibrary
from diffsims.generators.diffraction_generator import DiffractionGenerator
from diffsims.generators.library_generator import DiffractionLibraryGenerator
from diffsims.generators.rotation_list_generators import get_beam_directions_grid
from pyxem.utils import indexation_utils as iutls
from pyxem.signals.indexation_results import results_dict_to_crystal_map
from pyxem.utils import plotting_utils as putls
def do_template_matching(datapath,cifpath,res,transform,diffraction_calibration):
    dp = hs.load(datapath)
    print("Done Loading")
    resolution = res
    grid_cub = get_beam_directions_grid("cubic", resolution, mesh="spherified_cube_edge")

    structure_matrix = diffpy.structure.loadStructure(cifpath)
    half_shape = (dp.data.shape[-2]//2, dp.data.shape[-1]//2)

    reciprocal_radius = np.sqrt(half_shape[0]**2 + half_shape[1]**2)*diffraction_calibration

    diff_gen = DiffractionGenerator(accelerating_voltage=200,
                                    precession_angle=1,
                                    scattering_params=None,
                                    shape_factor_model="linear",
                                    minimum_intensity=0.1,
                                    )

    lib_gen = DiffractionLibraryGenerator(diff_gen)

    library_phases = StructureLibrary(["Cu"], [structure_matrix], [grid_cub])

    diff_lib = lib_gen.get_diffraction_library(library_phases,
                                               calibration=diffraction_calibration,
                                               reciprocal_radius=reciprocal_radius,
                                               half_shape=half_shape,
                                               with_direct_beam=False,
                                               max_excitation_error=0.06)
    print("Done Simulating")

    frac_keep = 1
    n_best = 20
    n_keep = None

    delta_r = 0.5
    delta_theta = 0.5
    max_r = None
    intensity_transform_function = transform
    direct_beam_position = None
    normalize_image = False
    normalize_templates = True

    result, phasedict = iutls.index_dataset_with_template_rotation(dp,
                                                        diff_lib,
                                                        phases = ["Cu"],
                                                        n_best = n_best,
                                                        frac_keep = frac_keep,
                                                        n_keep = n_keep,
                                                        delta_r = delta_r,
                                                        delta_theta = delta_theta,
                                                        max_r = None,
                                                        intensity_transform_function=intensity_transform_function,
                                                        normalize_images = normalize_image,
                                                        normalize_templates=normalize_templates,
                                                        target='gpu'
                                                        )

    xmap = results_dict_to_crystal_map(result,phasedict)
    xmap.phases[0].space_group = 225
    return xmap

In [ ]:
diffraction_calibration = 0.009381971999999999
# xmap = do_template_matching("Data/CuTwinTilt/SPEDCutilt0_bgred.hspy","Data/Cu_mp-30_conventional_standard.cif",0.5,template_log_func,diffraction_calibration)
# xmap = do_template_matching("Data/CuTwinTilt/20220305_163410/SPED_256x256x12_25x25_scanRot340_NBD_alpha5_spot0p5nm_CL12cm_1deg_100Hz_20ms.hspy","Data/Cu_mp-30_conventional_standard.cif",0.5,template_log_func,diffraction_calibration)
# xmap = do_template_matching("Data/CuTwinTilt/SPEDCutilt2_bgred.hspy","Data/Cu_mp-30_conventional_standard.cif",0.5,template_log_func,diffraction_calibration)

In [ ]:
xmap.phases.add_not_indexed()

In [ ]:
xmap0['not_indexed'].phase_id = 0

In [ ]:
xmap[xmap.correlation[:,0] < 0.5].phase_id = -1

In [ ]:
xmap[xmap.correlation[:,0] > 2.3]

In [ ]:
xmap

In [ ]:
from orix import plot
from orix.crystal_map import CrystalMap, Phase, PhaseList
from orix.io import load, save
from orix.quaternion import Orientation, Rotation, symmetry
from orix.vector import Vector3d, AxAngle, Miller

In [ ]:
xmap = load("Data/CuTwinTiltCu0xmapBGRED.h5") 

In [ ]:
ckey_m3m = plot.IPFColorKeyTSL(xmap.phases["Cu"].point_group, direction=Vector3d.xvector())
rgb_Cux = ckey_m3m.orientation2color(xmap["Cu"].orientations)
ckey_m3m = plot.IPFColorKeyTSL(xmap.phases["Cu"].point_group, direction=Vector3d.yvector())
rgb_Cuy = ckey_m3m.orientation2color(xmap["Cu"].orientations)
ckey_m3m = plot.IPFColorKeyTSL(xmap.phases["Cu"].point_group, direction=Vector3d.zvector())
rgb_Cuz = ckey_m3m.orientation2color(xmap["Cu"].orientations)

In [ ]:
fig = plt.figure()

ax0 = fig.add_subplot(311, projection="plot_map")
ax0.tick_params(labelleft=False,labelbottom=False, length=0)
ax1 = fig.add_subplot(312, projection="plot_map")
ax1.tick_params(labelleft=False,labelbottom=False, length=0)
ax2 = fig.add_subplot(313, projection="plot_map")
ax2.tick_params(labelleft=False,labelbottom=False, length=0)
ax0.set_title("X")
ax1.set_title("Y")
ax2.set_title("Z")

ax0.plot_map(xmap["Cu"], rgb_Cux, scalebar = False)
ax1.plot_map(xmap["Cu"], rgb_Cuy, scalebar = False)
ax2.plot_map(xmap["Cu"], rgb_Cuz, scalebar = False)

In [ ]:
fig.savefig("orimapCu.png", bbox_inches="tight", pad_inches=0)

In [ ]:
xmap['Cu'].orientations.scatter('axangle')

In [ ]:
xmap.plot(value = xmap.correlation[:,0],scalebar = False)

In [ ]:
save(
    filename="Data/CuTwinTiltCu2xmapBGRED.h5",
    object2write=xmap,
    overwrite=True,  # Default is False
)

In [ ]:
ipfkey = plot.IPFColorKeyTSL(symmetry.Oh)
ipfkey.plot()

In [ ]:
ori.scatter(projection='ipf',direction=Vector3d((0,0,1)))

In [ ]:
px = 200
py = 35
index = np.ravel_multi_index((py,px),xmap.shape)

ori = xmap['Cu'].orientations[index]

In [ ]:
simulations = diff_lib["Cu"]['simulations']
#scan coordinate to check
px = 160
py = 30
# which solution to plot
n_sol = 0

index = np.ravel_multi_index((py,px),xmap.shape)
# query the necessary info from the solution
sim_sol_index = xmap.template_index[index,0]
mirrored_sol = xmap.mirrored_template[index,0]
in_plane_angle = np.rad2deg(xmap.rotations[index,0].to_euler())[0][0]
# # query the appropriate template
sim_sol = simulations[sim_sol_index]



fig = plt.figure()
ax0 = fig.add_subplot(121, projection="plot_map")
ax1 = fig.add_subplot(122)
ax0.set_title("Y")
ax0.plot_map(xmap['Cu'],rgb_Cuy)
ax0.scatter(px,py, marker = 'X', c = 'g', s = 30)
putls.plot_template_over_pattern(dpcu0.inav[px, py].data,
                                 sim_sol,
                                 ax = ax1,
                                 in_plane_angle=in_plane_angle,
                                 coordinate_system = "cartesian", 
                                 size_factor = 10,
                                 vmax=30,
                                 max_r = 200,
                                 mirrored_template=mirrored_sol,
                                 find_direct_beam=True,
                                 cmap = "inferno",
                                 marker_color = "green"
                                )

In [ ]:
xmap0 = load("Data/CuTwinTiltCu0xmapBGRED.h5") 
xmap1 = load("Data/CuTwinTiltCu1xmapBGRED.h5") 
xmap2 = load("Data/CuTwinTiltCu2xmapBGRED.h5") 
xmap3 = load("Data/CuTwinTiltCu3xmapBGRED.h5") 
xmap4 = load("Data/CuTwinTiltCu4xmapBGRED.h5") 

In [ ]:
xmap['Cu'].plot(value = xmap.correlation[:,0], scalebar = False, colorbar = True)

In [ ]:
orifirst = xmap['Cu'].orientations
store = np.zeros((xmap['Cu'].size,15))
normcorrs = xmap['Cu'].correlation[:,:]/np.linalg.norm(xmap['Cu'].correlation[:,:]) 
for i in range (15):
    corr_weight = normcorrs[:,i]/normcorrs[:,0]
    # corr_weight = 1 # for unweighted, not recommened
    orinext = Orientation(xmap['Cu'].rotations[:,i],symmetry=symmetry.Oh)
    store[:,i] = np.degrees((orinext - orifirst).angle.data) * abs(corr_weight)

mean_misori_weighted = np.mean(store[:,1::], axis = 1)

In [ ]:
xmap['Cu'].plot(value = mean_misori_weighted,scalebar = False,vmin = 0, vmax = 10, colorbar = True)

In [ ]:
normcorrs = xmap['Cu'].correlation[:,:]/np.linalg.norm(xmap['Cu'].correlation[:,:]) 
reliability = normcorrs[:,0] - normcorrs[:,1] 

In [ ]:
xmap['Cu'].plot(value = reliability,scalebar = False, colorbar = True)

In [ ]:
xmapdict = [xmap0, xmap1,xmap2,xmap3,xmap4]
means = np.zeros((2,5,4))
for i in range(5):
    misred = xmapdict[i]['Cu'].orientations - xmapdict[i][20,165].orientations
    misblu = xmapdict[i]['Cu'].orientations - xmapdict[i][30,210].orientations

    meanred = xmapdict[i][np.rad2deg(misred.angle.data) < 2].orientations.mean()
    meanblu = xmapdict[i][np.rad2deg(misblu.angle.data) < 2].orientations.mean()
    
    meanred.symmetry = symmetry.Oh
    meanblu.symmetry = symmetry.Oh
    
    means[0,i] = meanred.data
    means[1,i] = meanblu.data

    mistwin = np.rad2deg((meanred - meanblu).angle.data)
    print(mistwin)

In [ ]:
meansori = Orientation(means)

In [ ]:
meansori.symmetry = symmetry.Oh

In [ ]:
mistilt = np.rad2deg((meansori[0,0] - meansori[0,1]).angle.data)
print(mistilt)
mistilt = np.rad2deg((meansori[0,1] - meansori[0,2]).angle.data)
print(mistilt)
mistilt = np.rad2deg((meansori[0,0] - meansori[0,3]).angle.data)
print(mistilt)
mistilt = np.rad2deg((meansori[0,3] - meansori[0,4]).angle.data)
print(mistilt)

In [ ]:
mistilt = np.rad2deg((meansori[1,0] - meansori[1,1]).angle.data)
print(mistilt)
mistilt = np.rad2deg((meansori[1,1] - meansori[1,2]).angle.data)
print(mistilt)
mistilt = np.rad2deg((meansori[1,0] - meansori[1,3]).angle.data)
print(mistilt)
mistilt = np.rad2deg((meansori[1,3] - meansori[1,4]).angle.data)
print(mistilt)

In [ ]:
meanOris = Orientation(means, symmetry= symmetry.Oh)

In [ ]:
misred = xmap4['Cu'].orientations - xmap4[30,160].orientations
misblu = xmap4['Cu'].orientations - xmap4[30,210].orientations

meanred = xmap4[np.rad2deg(misred.angle.data) < 2].orientations.mean()
meanblu = xmap4[np.rad2deg(misblu.angle.data) < 2].orientations.mean()

meanred.symmetry = symmetry.Oh
meanblu.symmetry = symmetry.Oh

mistwin = np.rad2deg((meanred - meanblu).angle.data)
print(meanred)
print(meanblu)
print(mistwin)

In [ ]:
np.rad2deg((xmapdict[4][30,160].orientations - xmapdict[4][30,210].orientations).angle.data)

In [ ]:
wherered = np.where(np.rad2deg(misred.angle.data) < 2)[0]
whereblu = np.where(np.rad2deg(misblu.angle.data) < 2)[0]

In [ ]:
test2 =xmap0[np.rad2deg(xmap0['Cu'].orientations.angle_with(xmap0[30,160].orientations).data) < 2].orientations.map_into_symmetry_reduced_zone().mean()

In [ ]:
test1 =xmap[np.rad2deg(misred.angle.data) < 2].orientations.mean()

In [ ]:
np.rad2deg(test1.angle_with(test2).data)

In [ ]:
ckey_m3m = plot.IPFColorKeyTSL(xmap0.phases["Cu"].point_group, direction=Vector3d.xvector())
rgb_Cux = ckey_m3m.orientation2color(xmap0["Cu"].orientations[whereblu])
ckey_m3m = plot.IPFColorKeyTSL(xmap0.phases["Cu"].point_group, direction=Vector3d.yvector())
rgb_Cuy = ckey_m3m.orientation2color(xmap0["Cu"].orientations[whereblu])
ckey_m3m = plot.IPFColorKeyTSL(xmap0.phases["Cu"].point_group, direction=Vector3d.zvector())
rgb_Cuz = ckey_m3m.orientation2color(xmap0["Cu"].orientations[whereblu])

In [ ]:
fig = plt.figure()
ax0 = fig.add_subplot(311, projection="plot_map")
ax1 = fig.add_subplot(312, projection="plot_map")
ax2 = fig.add_subplot(313, projection="plot_map")
ax0.set_title("X")
ax1.set_title("Y")
ax2.set_title("Z")
ax0.plot_map(xmap[np.rad2deg(misblu.angle.data) < 2], rgb_Cux, scalebar = False)
ax1.plot_map(xmap[np.rad2deg(misblu.angle.data) < 2], rgb_Cuy, scalebar = False)
ax2.plot_map(xmap[np.rad2deg(misblu.angle.data) < 2], rgb_Cuz, scalebar = False)

In [ ]:
trueorimeans[:,0]

In [ ]:
things = cluster_means[1:3].conj.outer(m111)

In [ ]:
things = ((trueorimeans[1,:].conj) * Miller((1,0,1)))

In [ ]:
things = ((trueorimeans[:,0].conj).outer(m111))

In [ ]:
things

In [ ]:
colors2 = ["C0", "C1","C0","C1","C0", "C1","C0","C1","C0", "C1","C0","C1","C0", "C1","C0","C1"]

In [ ]:
things.scatter(grid = True, grid_resolution= (5,1),c = ('b','g','g','r','r'))

In [ ]:
trueorimeans[0,:].scatter(projection= 'ipf', c = ('b','g','g','r','r'))

In [ ]:
xmap0['Cu'].orientations[np.indices(xmap0.shape)[0,40,33]]

In [ ]:
xmap0['Cu'].orientations[5120:8960]

In [ ]:
ori = Orientation(xmap1.rotations[5120:8960][:,0], symmetry=symmetry.Oh)

In [ ]:
D2 = ori.get_distance_matrix(lazy = True).data

In [ ]:
D2 = D2.reshape(ori.size, ori.size).data

In [ ]:
from sklearn.cluster import DBSCAN
dbscan = DBSCAN(eps=0.025, min_samples=30, metric="precomputed").fit(D2)
print("Labels:", np.unique(dbscan.labels_))

In [ ]:
from skimage.color import label2rgb
from matplotlib.colors import to_rgb
color_names = [to_rgb(f"C{i}") for i in range(12)]  # ['C0', 'C1', ...]

# colors_naive = label2rgb(dbscan_naive.labels_, colors=color_names, bg_label=-1)
colors = label2rgb(dbscan.labels_, colors=color_names, bg_label=-1)



In [ ]:
unique_labels, all_cluster_sizes = np.unique(dbscan.labels_, return_counts=True)
print("Labels:", unique_labels)

all_labels = dbscan.labels_.reshape(ori.shape)
n_clusters = unique_labels.size - 1
print("Number of clusters:", n_clusters)

In [ ]:
unique_cluster_labels = unique_labels[1:]  # Without the "no-cluster" label -1
cluster_sizes = all_cluster_sizes[1:]

q_mean = [ori[all_labels == l].mean() for l in unique_cluster_labels]
cluster_means = Orientation.stack(q_mean).flatten()

# Map into the fundamental zone
cluster_means.symmetry = symmetry.Oh
# cluster_means = cluster_means.map_into_symmetry_reduced_zone()
cluster_means

In [ ]:
np.rad2deg((cluster_means[0]- cluster_means[1]).angle.data)

In [ ]:
orimeans = np.zeros((2,5,4))

In [ ]:
trueorimeans = Orientation(orimeans, symmetry=symmetry.Oh)

In [ ]:
orimeans[0,0] = cluster_means[0].data
orimeans[1,0] = cluster_means[1].data

In [ ]:
orimeans[0,1] = cluster_means[0].data
orimeans[1,1] = cluster_means[1].data

In [ ]:
orimeans[0,2] = cluster_means[1].data
orimeans[1,2] = cluster_means[2].data

In [ ]:
orimeans[0,3] = cluster_means[0].data
orimeans[1,3] = cluster_means[1].data

In [ ]:
orimeans[0,4] = cluster_means[0].data
orimeans[1,4] = cluster_means[1].data

In [ ]:
import pickle
filename = "trueclusteredmeans.pickle"
pickling_on = open("trueclusteredmeans.pickle","wb")
pickle.dump(trueorimeans, pickling_on)
pickling_on.close()

In [ ]:
import pickle
pickle_off = open("trueclusteredmeans.pickle", 'rb')
trueorimeans = pickle.load(pickle_off)

In [ ]:
cindex = np.linspace(1, 2, 5)
trueorimeans[0,:].scatter(projection= 'ipf', c = ('b','g','g','y','r'), s = 50 )
trueorimeans[1,:].scatter(projection= 'ipf', c = ('b','g','g','y','r'), s = 50 )

In [ ]:
for i in range(5):
    print(np.rad2deg((trueorimeans[0,i] - trueorimeans[1,i]).angle.data))

In [ ]:
for i in range(4):
    print(np.rad2deg((trueorimeans[0,i] - trueorimeans[0,i+1]).angle.data))

In [ ]:
for i in range(4):
    print(np.rad2deg((trueorimeans[1,i] - trueorimeans[1,i+1]).angle.data))

In [ ]:
ori_recentered = (~cluster_means[0]) * ori

# Map into the fundamental zone
ori_recentered.symmetry = symmetry.Oh
# ori_recentered = ori_recentered.map_into_symmetry_reduced_zone()

cluster_means_recentered = Orientation.stack(
    [ori_recentered[all_labels == l].mean() for l in unique_cluster_labels]
).flatten()
cluster_means_recentered

In [ ]:
cluster_means_recentered_axangle = AxAngle.from_rotation(cluster_means_recentered)
cluster_means_recentered_axangle.axis

In [ ]:
colors = [to_rgb(f"C{i}") for i in range(cluster_means_recentered_axangle.size)]
labels_rgb = label2rgb(all_labels, colors=colors, bg_label=-1)

lines = [((0, 0, 0), tuple(cm)) for cm in cluster_means_recentered_axangle.data]

In [ ]:
cluster_sizes_scaled = 5000 * cluster_sizes / cluster_sizes.max()
fig, ax = plt.subplots(figsize=(5, 5), subplot_kw=dict(projection="ipf", symmetry=symmetry.Oh))
ax.scatter(
    cluster_means.axis, c=colors, s=cluster_sizes_scaled, alpha=0.5, ec="k"
)

In [ ]:
from mpl_toolkits.mplot3d.art3d import Line3DCollection
from matplotlib.lines import Line2D
wireframe_kwargs = dict(color="black", linewidth=0.5, alpha=0.1, rcount=181, ccount=361)
fig = ori_recentered.scatter(
    projection="axangle",
    wireframe_kwargs=wireframe_kwargs,
    c=labels_rgb.reshape(-1, 3),
    s=1,
    return_figure=True
)
ax = fig.axes[0]
ax.view_init(elev=90, azim=-30)
ax.add_collection3d(Line3DCollection(lines, colors=colors))

handle_kwds = dict(marker="o", color="none", markersize=10)
handles = []
for i in range(n_clusters):
    line = Line2D([0], [0], label=i + 1, markerfacecolor=colors[i], **handle_kwds)
    handles.append(line)
ax.legend(
    handles=handles,
    loc="lower right",
    ncol=2,
    numpoints=1,
    labelspacing=0.15,
    columnspacing=0.15,
    handletextpad=0.05
);

In [ ]:
fig3 = plt.figure()
ax3 = fig3.add_subplot()
ax3.imshow(labels_rgb)
ax3.axis("off");

In [ ]:
labels_rgb = labels_rgb.reshape((15,256,3))

In [ ]:
labels_rgb.shape